In [1]:
import numpy as np
import matplotlib.pyplot as plt
from seaborn import set_palette
import pickle
flatui = ["#3498db", "#9b59b6", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
set_palette(flatui)
%matplotlib inline
%config InlineBackend.figure_format = "retina"
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.color'] = 'gray'
plt.rcParams['grid.linewidth'] = 0.25
plt.rcParams['grid.alpha'] = 0.2
plt.style.use('seaborn-talk')

from scipy.signal import stft, istft, get_window
from IPython.display import Audio
from tqdm import tnrange, tqdm_notebook
from dlbeamformer_utilities import compute_steering_vectors_single_frequency,\
    compute_steering_vectors, simulate_multichannel_tf, compute_sinr,\
    compute_mvdr_tf_beamformers, check_distortless_constraint,\
    compute_mvndr_tf_beamformers, compute_lcmv_tf_beamformers,\
    compute_null_controlling_tf_beamformers
from dlbeamformers import BaseDLBeamformer, DLBeamformer
random_seed = 0

In [2]:
import os
from os import listdir
from os.path import join
datapath = "CMU_ARCTIC/cmu_us_bdl_arctic/wav"
train_data_folder = join(datapath, 'train')
test_data_folder = join(datapath, 'test')

from scipy.io import wavfile
from IPython.display import Audio
train_data = []
test_data = []
train_data_filenames = [f for f in listdir(train_data_folder) if os.path.isfile( join(train_data_folder, f))]
test_data_filenames = [f for f in listdir(test_data_folder) if os.path.isfile( join(test_data_folder, f))]

for i_train_data_filename in range(len(train_data_filenames)):
    f_path = join(train_data_folder, train_data_filenames[i_train_data_filename])
    if f_path.endswith('.wav'):
        sampling_frequency, train_data_example = wavfile.read(f_path)
    train_data.append(train_data_example)
    
for i_test_data_filename in range(len(test_data_filenames)):
    f_path = join(test_data_folder, test_data_filenames[i_test_data_filename])
    if f_path.endswith('.wav'):
        sampling_frequency, test_data_example = wavfile.read(f_path)
    test_data.append(test_data_example)

In [3]:
# Microphone positions
pos_x = np.arange(-0.8, 0.8+1e-6, 0.2)
n_mics = len(pos_x)
pos_y = np.zeros(n_mics)
pos_z = np.zeros(n_mics)
array_geometry = np.row_stack((pos_x, pos_y, pos_z))
array_geometry.shape

(3, 9)

In [4]:
from configparser import ConfigParser
config = ConfigParser()
config.read('config.INI');
params = config['PARAMS']
sampling_frequency = int(params['sampling_frequency'])
n_samples_per_frame = int(params['n_samples_per_frame'])
n_fft_bins = (int) (n_samples_per_frame / 2) 
hop_size = (int) (n_samples_per_frame / 2)
stft_window_name = params['stft_window_name']
stft_window = get_window("hann", n_samples_per_frame)
stft_params = {
    "n_samples_per_frame": n_samples_per_frame,
    "n_fft_bins": n_fft_bins,
    "hop_size": hop_size,
    "window": stft_window
}

In [ ]:
# Source angles
theta_s = np.array([-10]) # [degree]
phi_s = np.array([0]) # [degree]

# Angle grids
theta_grid = np.arange(-90, 90+1e-6, 0.1) # [degree]
phi_grid = np.array([0]) # [degree]

# Steering vectors
steering_vectors = compute_steering_vectors(array_geometry, sampling_frequency=sampling_frequency, 
                                    n_fft=n_fft_bins, theta_grid=theta_grid, phi_grid=phi_grid)
source_steering_vectors = compute_steering_vectors(array_geometry, sampling_frequency, n_fft_bins, 
    np.array([theta_s[0]]), np.array([phi_s[0]]))
vs = compute_steering_vectors(array_geometry, sampling_frequency, n_fft_bins, 
    theta_s, phi_s)

### DLBeamformer

In [ ]:
source_angles = [
    {
        "theta": np.array([-10]),
        "phi": np.array([0])
    }
]
angle_grid = {
    "theta": theta_grid,
    "phi": phi_grid
}
dlbeamformer = DLBeamformer(array_geometry, sampling_frequency, source_angles,
                           stft_params, angle_grid)
assert np.allclose(vs, dlbeamformer.source_steering_vectors[0])
assert np.allclose(steering_vectors, dlbeamformer.steering_vectors)

In [ ]:
np.unique(np.array([1, 2, 3, 3, 4, 3, -1, -2]))

array([-2, -1,  1,  2,  3,  4])

### Training data for MVDR baseline dictionary

In [ ]:
np.random.seed(random_seed)
n_interference_list = [1, 2]

azimuth_step = 30
training_thetas = list(np.arange(-90, 90, azimuth_step))
training_phis = [0]

import itertools
training_interference_data = []
training_noise_interference_data = []
np.random.seed(random_seed)

for i_n_interference in tqdm_notebook(range(len(n_interference_list)), desc="Interference number"):
    n_interferences = n_interference_list[i_n_interference]
    interferences_params = []
    for i_interference in range(n_interferences):
        interference_params = list(itertools.product(*[training_thetas, training_phis]))
        interferences_params.append(interference_params)
    interferences_param_sets = list(itertools.product(*interferences_params))

    for i_param_set in tqdm_notebook(range(len(interferences_param_sets)), desc="Parameter set"):    
        param_set = interferences_param_sets[i_param_set]
        n_training_samples = 1
        for i_training_sample in range(n_training_samples):
            interference_signals = []
            for i_interference in range(len(param_set)):
                interference_signal = train_data[np.random.choice(len(train_data))]
                interference_signals.append(interference_signal)                
            interference_n_samples = min([len(signal) for signal in interference_signals])
            
            interference_tf_multichannel_list = []
            for i_interference in range(len(param_set)):
                interference_signals[i_interference] = (interference_signals[i_interference])[0:interference_n_samples]
                interference_theta, interference_phi = param_set[i_interference]
                interference_theta += 0*np.random.uniform()
                interference_tf_multichannel = simulate_multichannel_tf(array_geometry, interference_signal, 
                        np.array([interference_theta]), np.array([interference_phi]),
                        sampling_frequency, stft_params)
                interference_tf_multichannel_list.append(interference_tf_multichannel)
            
            training_interference_data.append(sum(interference_tf_multichannel_list))
# training_tf_filename = "CMU_ARCTIC_tf_training_data_azimuth_step_{}_trainning_samples_{}.pkl".format(
#     azimuth_step, n_training_samples
# )
# training_tf_path = "/data/dung/dictionary-beamforming-speech-enhancement/tf_training_data"
# training_tf_filepath = os.path.join(training_tf_path, training_tf_filename)
# with open(training_tf_filepath, 'wb') as output:
#     pickle.dump(training_interference_data, output, pickle.HIGHEST_PROTOCOL)            

### Training data for null-controlling DL Beamformers

In [ ]:
np.random.seed(random_seed)
n_interference_list = [1, 2]

azimuth_step = 30
training_thetas = list(np.arange(-90, 90, azimuth_step))
training_phis = [0]

import itertools
nc_training_interference_data = []
training_noise_interference_data = []
np.random.seed(random_seed)

for i_n_interference in tqdm_notebook(range(len(n_interference_list)), desc="Interference number"):
    n_interferences = n_interference_list[i_n_interference]
    interferences_params = []
    for i_interference in range(n_interferences):
        interference_params = list(itertools.product(*[training_thetas, training_phis]))
        interferences_params.append(interference_params)
    interferences_param_sets = list(itertools.product(*interferences_params))

    for i_param_set in tqdm_notebook(range(len(interferences_param_sets)), desc="Parameter set"):    
        param_set = interferences_param_sets[i_param_set]
        training_param_dict = {
            "theta": np.asarray(param_set).transpose()[0],
            "phi": np.asarray(param_set).transpose()[1]
            }
        n_training_samples = 100
        for i_training_sample in range(n_training_samples):
            interference_signals = []
            for i_interference in range(len(param_set)):
                interference_signal = train_data[np.random.choice(len(train_data))]
                interference_signals.append(interference_signal)                
            interference_n_samples = min([len(signal) for signal in interference_signals])
            
            interference_tf_multichannel_list = []
            for i_interference in range(len(param_set)):
                interference_signals[i_interference] = (interference_signals[i_interference])[0:interference_n_samples]
                interference_theta, interference_phi = param_set[i_interference]
                interference_theta += 0*np.random.uniform()
                interference_tf_multichannel = simulate_multichannel_tf(array_geometry, interference_signal, 
                        np.array([interference_theta]), np.array([interference_phi]),
                        sampling_frequency, stft_params)
                interference_tf_multichannel_list.append(interference_tf_multichannel)
            
            nc_training_interference_data.append((sum(interference_tf_multichannel_list), training_param_dict))

In [ ]:
l = []
desired_null_width = 5
for th in nc_training_interference_data[-1][1]["theta"]:
#     th = training_interference_data[30][1]["theta"][i]
    l.append(np.arange(th-desired_null_width/2, th+desired_null_width/2, 0.1))
assert np.allclose(np.concatenate(l), 
                   dlbeamformer._compute_null_angle_ranges(
                   nc_training_interference_data[-1][1]["theta"],
                   desired_null_width))
# v = dlbeamformer._compute_weights(training_interference_data, desired_null_width)
# alps = np.concatenate(l)
# phs = phi_grid
# nsv = compute_steering_vectors(array_geometry, sampling_frequency=sampling_frequency, 
#                                     n_fft=n_fft_bins, theta_grid=np.unique(alps), phi_grid=np.unique(phs) )
# assert np.allclose(v, nsv)
null_constraint_threshold = 0.01
dlbeamformer.fit(nc_training_interference_data, desired_null_width, 
                null_constraint_threshold, eigenvalue_percentage_threshold=0.99)

Training sample:  53%|█████▎    | 2208/4200 [04:37<04:10,  7.95it/s]

In [ ]:
dictionary = BaseDLBeamformer(source_steering_vectors[:, 0, 0, :])
dictionary.fit(training_interference_data);

In [ ]:
# dict_filename = "baseline_dl_azimuth_step_{}_trainning_samples_{}.pkl".format(
#     azimuth_step, n_training_samples
# )
# train_models_path = "/data/dung/dictionary-beamforming-speech-enhancement/trained_models"
# dict_filepath = os.path.join(train_models_path, dict_filename)
# with open(dict_filepath, 'wb') as output:
#     pickle.dump(dictionary, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
# Simulate received multichannel signal
np.random.seed(0)

# Test signals    
n_test_samples = 40000
source_signal = test_data[0][0:n_test_samples]
interference_signals = []
for i_interference in range(2):
    interference_signals.append(test_data[i_interference+1][0:n_test_samples])
    
# Interference angles
thetas_i = np.array([32]) # [degree]
phis_i =np.array([0]) # [degree]

time_samples = np.arange(0, n_test_samples)/sampling_frequency    

source_stft_multichannel = simulate_multichannel_tf(array_geometry, source_signal, theta_s, phi_s,
                                sampling_frequency, stft_params)

received_stft_multichannel = np.zeros(source_stft_multichannel.shape, dtype=np.complex64)

received_stft_multichannel += source_stft_multichannel

interference_stfts_multichannel_list = []

for i_angle in range(len(thetas_i)):
    interference_signal = interference_signals[i_angle]
    
    interference_stft_multichannel = simulate_multichannel_tf(array_geometry, interference_signal, 
                                        np.array([thetas_i[i_angle]]), np.array([phis_i[i_angle]]),
                                        sampling_frequency, stft_params)

    interference_stfts_multichannel_list.append(interference_stft_multichannel)
    
    received_stft_multichannel += interference_stft_multichannel
    
interference_stfts_multichannel = sum(interference_stfts_multichannel_list)

In [ ]:
# Listen to the simulated received signal
t, ss = istft(np.real(received_stft_multichannel[:, 0, :]), fs=sampling_frequency, window=stft_window,
                         nperseg=n_samples_per_frame, noverlap=n_samples_per_frame-hop_size,
                         nfft=n_samples_per_frame, boundary=True)

plt.imshow(10*np.log10(np.abs(received_stft_multichannel[:, 0, :])), origin='lower', aspect='auto')
input_sinr = compute_sinr(source_stft_multichannel, interference_stft_multichannel)
print("Input SINR: {:.2f} dB".format(input_sinr))
Audio(np.real(ss), rate=sampling_frequency, autoplay=True)

### Beamformers and outputs

In [ ]:
# Delay-sum beamformer
ds_tf_beamformers = 1./n_mics * source_steering_vectors
ds_tf_out = np.zeros((n_fft_bins, received_stft_multichannel.shape[2]), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    ds_tf_out[i_fft_bin] = ds_tf_beamformers[i_fft_bin, 0, 0].transpose().conjugate().dot(received_stft_multichannel[i_fft_bin])
t, ds_out_2 = istft(ds_tf_out, fs=sampling_frequency, window=stft_window,
                         nperseg=n_samples_per_frame, noverlap=n_samples_per_frame-hop_size,
                         nfft=n_samples_per_frame, boundary=True)    

plt.imshow(10*np.log10(np.abs(ds_tf_out)), origin='lower', aspect='auto')
ds_out_sinr = compute_sinr(source_stft_multichannel, interference_stft_multichannel, ds_tf_beamformers[:, 0, 0, :])
print("Output SINR: {:.2f} dB".format(ds_out_sinr[0][0]))
Audio(np.real(ds_out_2), rate=sampling_frequency, autoplay=True)

In [ ]:
mpdr_tf_beamformers = compute_mvdr_tf_beamformers(source_steering_vectors[:, 0, 0, :], received_stft_multichannel)
mpdr_tf_out = np.zeros((n_fft_bins, received_stft_multichannel.shape[2]), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    mpdr_tf_out[i_fft_bin] = mpdr_tf_beamformers[i_fft_bin].transpose().conjugate().dot(received_stft_multichannel[i_fft_bin])
t, mpdr_out = istft(mpdr_tf_out, fs=sampling_frequency, window=stft_window,
                         nperseg=n_samples_per_frame, noverlap=n_samples_per_frame-hop_size,
                         nfft=n_samples_per_frame, boundary=True)

plt.imshow(10*np.log10(np.abs(mpdr_tf_out)), origin='lower', aspect='auto')
mpdr_out_sinr = compute_sinr(source_stft_multichannel, interference_stft_multichannel, mpdr_tf_beamformers)
print("MPDR output SINR: {:.2f} dB".format(mpdr_out_sinr[0][0]))
Audio(np.real(mpdr_out), rate=sampling_frequency, autoplay=True)

In [ ]:
mvdr_tf_beamformers = compute_mvdr_tf_beamformers(source_steering_vectors[:, 0, 0, :], interference_stfts_multichannel)
mvdr_tf_out = np.zeros((n_fft_bins, received_stft_multichannel.shape[2]), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    mvdr_tf_out[i_fft_bin] = mvdr_tf_beamformers[i_fft_bin].transpose().conjugate().dot(received_stft_multichannel[i_fft_bin])
t, mvdr_out_2 = istft(mvdr_tf_out, fs=sampling_frequency, window=stft_window,
                         nperseg=n_samples_per_frame, noverlap=n_samples_per_frame-hop_size,
                         nfft=n_samples_per_frame, boundary=True)

plt.imshow(10*np.log10(np.abs(mvdr_tf_out)), origin='lower', aspect='auto')
mvdr_out_sinr = compute_sinr(source_stft_multichannel, interference_stft_multichannel, mvdr_tf_beamformers)
print("MVDR output SINR: {:.2f} dB".format(mvdr_out_sinr[0][0]))
Audio(np.real(mvdr_out_2), rate=sampling_frequency, autoplay=True)

In [ ]:
theta_nulling = np.array([27, 29, 31, 33, 35])
theta_all = np.hstack((theta_s, theta_nulling))
phi_all = np.zeros([1])
lcmv_constraint_vector = np.zeros(theta_nulling.shape[0] + 1)
lcmv_constraint_vector[0] = 1
# theta_all = theta_s
# lcmv_constraint_vector = np.array([1])
lcmv_steering_vectors = compute_steering_vectors(array_geometry, sampling_frequency=sampling_frequency, 
                                    n_fft=n_fft_bins, theta_grid=theta_all, phi_grid=phi_all) 
C = np.transpose(lcmv_steering_vectors[:, :, 0, :], (0, 2, 1))
lcmv_tf_beamformers = compute_lcmv_tf_beamformers(C, interference_stft_multichannel, lcmv_constraint_vector)
lcmv_tf_out = np.zeros((n_fft_bins, received_stft_multichannel.shape[2]), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    lcmv_tf_out[i_fft_bin] = lcmv_tf_beamformers[i_fft_bin].transpose().conjugate().dot(received_stft_multichannel[i_fft_bin])
t, lcmv_out = istft(lcmv_tf_out, fs=sampling_frequency, window=stft_window,
                         nperseg=n_samples_per_frame, noverlap=n_samples_per_frame-hop_size,
                         nfft=n_samples_per_frame, boundary=True)

plt.imshow(10*np.log10(np.abs(lcmv_tf_out)), origin='lower', aspect='auto')
lcmv_out_sinr = compute_sinr(source_stft_multichannel, interference_stft_multichannel, lcmv_tf_beamformers)
print("LCMV output SINR: {:.2f} dB".format(lcmv_out_sinr[0][0]))
Audio(np.real(lcmv_out), rate=sampling_frequency, autoplay=True)

In [ ]:
# theta_nulling = np.hstack((np.arange(2.5, 7.5, .1), np.arange(27.5, 32.5, .1)))
theta_nulling = np.arange(27.5, 32.5, .1)
theta_all = np.hstack((theta_s, theta_nulling))
phi_all = np.zeros([1])

################################################################################
# SANITY CHECK: this should result in a beam pattern similar to that of MVDR
# theta_nulling = theta_s
################################################################################
null_steering_vectors = compute_steering_vectors(array_geometry, sampling_frequency=sampling_frequency, 
                                    n_fft=n_fft_bins, theta_grid=theta_nulling, phi_grid=phi_all) 
null_steering_vectors = np.transpose(null_steering_vectors[:, :, 0, :], (0, 2, 1))
print(source_steering_vectors.shape)
null_constraint_threshold = 0.01
nc_tf_beamformers = compute_null_controlling_tf_beamformers(source_steering_vectors[:, 0, 0, :], 
                null_steering_vectors, interference_stft_multichannel, 
                null_constraint_threshold, eigenvalue_percentage_threshold=0.99)
nc_tf_out = np.zeros((n_fft_bins, received_stft_multichannel.shape[2]), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    nc_tf_out[i_fft_bin] = nc_tf_beamformers[i_fft_bin].transpose().conjugate().dot(received_stft_multichannel[i_fft_bin])
t, nc_out = istft(nc_tf_out, fs=sampling_frequency, window=stft_window,
                         nperseg=n_samples_per_frame, noverlap=n_samples_per_frame-hop_size,
                         nfft=n_samples_per_frame, boundary=True)

plt.imshow(10*np.log10(np.abs(nc_tf_out)), origin='lower', aspect='auto')
nc_out_sinr = compute_sinr(source_stft_multichannel, interference_stft_multichannel, nc_tf_beamformers)
print("NC output SINR: {:.2f} dB".format(nc_out_sinr[0][0]))
Audio(np.real(lcmv_out), rate=sampling_frequency, autoplay=True)

In [ ]:
_, _, mvdr_stft = stft(np.real(mvdr_out_2), fs=sampling_frequency, window=stft_window,
                         nperseg=n_samples_per_frame, noverlap=n_samples_per_frame-hop_size,
                         nfft=n_samples_per_frame, padded=True)
t, s = istft(mvdr_stft, fs=sampling_frequency, window=stft_window,
                         nperseg=n_samples_per_frame, noverlap=n_samples_per_frame-hop_size,
                         nfft=n_samples_per_frame, boundary=True)
Audio(np.real(s), rate=sampling_frequency, autoplay=True)

In [ ]:
source_angle_index = 0
nc_dl_tf_beamformers, _ = dlbeamformer.choose_weights(source_angle_index, 
                            interference_stfts_multichannel)
nc_dl_tf_out = np.zeros((n_fft_bins, interference_stfts_multichannel.shape[2]), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    nc_dl_tf_out[i_fft_bin] = nc_dl_tf_beamformers[i_fft_bin].transpose().conjugate().dot(received_stft_multichannel[i_fft_bin])
t, nc_dl_out = istft(nc_dl_tf_out, fs=sampling_frequency, window=stft_window,
                         nperseg=n_samples_per_frame, noverlap=n_samples_per_frame-hop_size,
                         nfft=n_samples_per_frame, boundary=True)

plt.imshow(10*np.log10(np.abs(nc_dl_tf_out)), origin='lower', aspect='auto')
nc_dl_out_sinr = compute_sinr(source_stft_multichannel, interference_stft_multichannel, nc_dl_tf_beamformers)
print("Null controlling DL output SINR: {:.2f} dB".format(nc_dl_out_sinr[0][0]))
Audio(np.real(nc_dl_out), rate=sampling_frequency, autoplay=True)

In [ ]:
mvdr_dl_tf_beamformers, _ = dictionary.choose_weights(interference_stfts_multichannel)
mvdr_dl_tf_out = np.zeros((n_fft_bins, interference_stfts_multichannel.shape[2]), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    mvdr_dl_tf_out[i_fft_bin] = mvdr_dl_tf_beamformers[i_fft_bin].transpose().conjugate().dot(received_stft_multichannel[i_fft_bin])
t, mvdr_dl_out = istft(mvdr_dl_tf_out, fs=sampling_frequency, window=stft_window,
                         nperseg=n_samples_per_frame, noverlap=n_samples_per_frame-hop_size,
                         nfft=n_samples_per_frame, boundary=True)

plt.imshow(10*np.log10(np.abs(mvdr_dl_tf_out)), origin='lower', aspect='auto')
mvdr_dl_out_sinr = compute_sinr(source_stft_multichannel, interference_stft_multichannel, mvdr_dl_tf_beamformers)
print("MVDR DL output SINR: {:.2f} dB".format(mvdr_dl_out_sinr[0][0]))
Audio(np.real(mvdr_dl_out), rate=sampling_frequency, autoplay=True)

In [ ]:
mpdr_dl_tf_beamformers, _ = dictionary.choose_weights(received_stft_multichannel)
mpdr_dl_tf_out = np.zeros((n_fft_bins, received_stft_multichannel.shape[2]), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    mpdr_dl_tf_out[i_fft_bin] = mpdr_dl_tf_beamformers[i_fft_bin].transpose().conjugate().dot(received_stft_multichannel[i_fft_bin])
t, mpdr_dl_out = istft(mpdr_dl_tf_out, fs=sampling_frequency, window=stft_window,
                         nperseg=n_samples_per_frame, noverlap=n_samples_per_frame-hop_size,
                         nfft=n_samples_per_frame, boundary=True)

plt.imshow(10*np.log10(np.abs(mpdr_dl_tf_out)), origin='lower', aspect='auto')
mpdr_dl_out_sinr = compute_sinr(source_stft_multichannel, interference_stft_multichannel, mpdr_dl_tf_beamformers)
print("MPDR DL output SINR: {:.2f} dB".format(mpdr_dl_out_sinr[0][0]))
Audio(np.real(mpdr_dl_out), rate=sampling_frequency, autoplay=True)

In [ ]:
regularization_param = 1e4
mvndr_tf_beamformers = compute_mvndr_tf_beamformers(source_steering_vectors[:, 0, 0, :], 
                                            interference_stfts_multichannel, regularization_param)
mvndr_tf_out = np.zeros((n_fft_bins, received_stft_multichannel.shape[2]), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    mvndr_tf_out[i_fft_bin] = mvndr_tf_beamformers[i_fft_bin].transpose().conjugate().dot(received_stft_multichannel[i_fft_bin])
t, mvndr_out = istft(mvndr_tf_out, fs=sampling_frequency, window=stft_window,
                         nperseg=n_samples_per_frame, noverlap=n_samples_per_frame-hop_size,
                         nfft=n_samples_per_frame, boundary=True)

plt.imshow(10*np.log10(np.abs(mvndr_tf_out)), origin='lower', aspect='auto')
# mvndr_out_sinr = compute_sinr(source_stft_multichannel, interference_stft_multichannel, mvndr_tf_beamformers)
# print("MVNDR output SINR: {:.2f} dB".format(mvndr_out_sinr[0][0]))
Audio(np.real(mvndr_out), rate=sampling_frequency, autoplay=True)

### Beam pattern

In [ ]:
# Delay-sum beam pattern
ds_tf_beampattern = np.zeros((n_fft_bins, len(theta_grid), len(phi_grid)), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    for i_theta in range(len(theta_grid)):
        for i_phi in range(len(phi_grid)):
            scanning_steering_vector = steering_vectors[i_fft_bin][i_theta][i_phi]
            ds_tf_beampattern[i_fft_bin][i_theta][i_phi] = ds_tf_beamformers[i_fft_bin, 0, 0, :].transpose().conjugate().dot(
            scanning_steering_vector)

ds_tf_beampattern_normalized = np.abs(ds_tf_beampattern)/np.max(np.abs(ds_tf_beampattern))
ds_tf_beampattern_db = 20*np.log10(ds_tf_beampattern_normalized);

fig = plt.figure(figsize=(9, 6)); ax = fig.add_subplot(111)
i_frequencies = [10, 20, 30]
for i_f in i_frequencies:
    ax.plot(theta_grid, ds_tf_beampattern_db[i_f, :, 0], label="Beam pattern");
ax.axvline(x=theta_s[0], linestyle="--", color=flatui[1], label="Source angle");
for i_interference in range(0, len(thetas_i)):
    ax.axvline(x=thetas_i[i_interference], linestyle="--", color=flatui[i_interference+2], label="Interference angle");
ax.set_xlim(-90, 90);
ax.set_xlabel(r"$\theta$ [degree]"); ax.set_ylabel("Beam pattern [dB]");
ax.set_title("Delay-sum TF beam patterns")
ax.legend();

ds_ave_beampattern = np.mean(np.abs(ds_tf_beampattern[:, :, 0]), axis=0)
ds_ave_beampattern_normalized = ds_ave_beampattern / np.max(ds_ave_beampattern)
ds_ave_beampattern_normalized_db = 20*np.log10(ds_ave_beampattern_normalized)
fig = plt.figure(figsize=(9, 6)); ax = fig.add_subplot(111)
ax.plot(theta_grid, ds_ave_beampattern_normalized_db, label="Beam pattern");
ax.axvline(x=theta_s[0], linestyle="--", color=flatui[1], label="Source angle");
for i_interference in range(0, len(thetas_i)):
    ax.axvline(x=thetas_i[i_interference], linestyle="--", color=flatui[i_interference+2], label="Interference angle");
ax.set_xlim(-90, 90); ax.set_ylim(-15, 1)
ax.set_xlabel(r"$\theta$ [degree]"); ax.set_ylabel("Beam pattern [dB]");
ax.set_title("Delay-sum TF average beam pattern")
ax.legend();

In [ ]:
# MVDR beam pattern
mvdr_tf_beampattern = np.zeros((n_fft_bins, len(theta_grid), len(phi_grid)), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    for i_theta in range(len(theta_grid)):
        for i_phi in range(len(phi_grid)):
            scanning_steering_vector = steering_vectors[i_fft_bin][i_theta][i_phi]
            mvdr_tf_beampattern[i_fft_bin][i_theta][i_phi] = mvdr_tf_beamformers[i_fft_bin].transpose().conjugate().dot(
            scanning_steering_vector)

# mvdr_tf_beampattern_normalized = np.abs(mvdr_tf_beampattern)/np.max(np.abs(mvdr_tf_beampattern))
mvdr_tf_beampattern_db = 20*np.log10(np.abs(mvdr_tf_beampattern));

fig = plt.figure(figsize=(9, 6)); ax = fig.add_subplot(111)
i_frequencies = [10, 20, 30, 40, 50]
for i_f in i_frequencies:
    ax.plot(theta_grid, mvdr_tf_beampattern_db[i_f, :, 0], label="{:.0f} Hz".format(i_f/n_fft_bins*sampling_frequency));
ax.axvline(x=theta_s[0], linestyle="--", color=flatui[1], label="Source angle");
for i_interference in range(0, len(thetas_i)):
    ax.axvline(x=thetas_i[i_interference], linestyle="--", color=flatui[i_interference+2], label="Interference angle");
ax.set_xlim(-90, 90);
ax.set_xlabel(r"$\theta$ [degree]"); ax.set_ylabel("Beam pattern [dB]");
ax.set_title("MVDR TF beam patterns")
ax.legend();

mvdr_ave_beampattern = np.mean(np.abs(mvdr_tf_beampattern[:, :, 0]), axis=0)
mvdr_ave_beampattern_normalized = mvdr_ave_beampattern / np.max(mvdr_ave_beampattern)
mvdr_ave_beampattern_normalized_db = 20*np.log10(mvdr_ave_beampattern_normalized)
fig = plt.figure(figsize=(9, 6)); ax = fig.add_subplot(111)
ax.plot(theta_grid, mvdr_ave_beampattern_normalized_db, label="Beam pattern");
ax.axvline(x=theta_s[0], linestyle="--", color=flatui[1], label="Source angle");
for i_interference in range(0, len(thetas_i)):
    ax.axvline(x=thetas_i[i_interference], linestyle="--", color=flatui[i_interference+2], label="Interference angle");
ax.set_xlim(-90, 90); ax.set_ylim(-15, 1)
ax.set_xlabel(r"$\theta$ [degree]"); ax.set_ylabel("Beam pattern [dB]");
ax.set_title("MVDR TF average beam pattern")
ax.legend();

In [ ]:
# LCMV beam pattern
lcmv_tf_beampattern = np.zeros((n_fft_bins, len(theta_grid), len(phi_grid)), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    for i_theta in range(len(theta_grid)):
        for i_phi in range(len(phi_grid)):
            scanning_steering_vector = steering_vectors[i_fft_bin][i_theta][i_phi]
            lcmv_tf_beampattern[i_fft_bin][i_theta][i_phi] = lcmv_tf_beamformers[i_fft_bin].transpose().conjugate().dot(
            scanning_steering_vector)

# mvdr_tf_beampattern_normalized = np.abs(mvdr_tf_beampattern)/np.max(np.abs(mvdr_tf_beampattern))
lcmv_tf_beampattern_db = 20*np.log10(np.abs(lcmv_tf_beampattern));

fig = plt.figure(figsize=(9, 6)); ax = fig.add_subplot(111)
i_frequencies = [10, 30, 50]
for i_f in i_frequencies:
    ax.plot(theta_grid, lcmv_tf_beampattern_db[i_f, :, 0], label="{:.0f} Hz".format(i_f/n_fft_bins*sampling_frequency));
ax.axvline(x=theta_s[0], linestyle="--", color=flatui[1], label="Source angle");
for i_interference in range(0, len(thetas_i)):
    ax.axvline(x=thetas_i[i_interference], linestyle="--", color=flatui[i_interference+2], label="Interference angle");
ax.set_xlim(-90, 90);
ax.set_xlabel(r"$\theta$ [degree]"); ax.set_ylabel("Beam pattern [dB]");
ax.set_title("LCMV TF beam patterns")
ax.legend();

lcmv_ave_beampattern = np.mean(np.abs(lcmv_tf_beampattern[:, :, 0]), axis=0)
lcmv_ave_beampattern_normalized = lcmv_ave_beampattern / np.max(lcmv_ave_beampattern)
lcmv_ave_beampattern_normalized = lcmv_ave_beampattern
lcmv_ave_beampattern_normalized_db = 20*np.log10(lcmv_ave_beampattern_normalized)
fig = plt.figure(figsize=(9, 6)); ax = fig.add_subplot(111)
ax.plot(theta_grid, lcmv_ave_beampattern_normalized_db, label="Beam pattern");
ax.axvline(x=theta_s[0], linestyle="--", color=flatui[1], label="Source angle");
for i_interference in range(0, len(thetas_i)):
    ax.axvline(x=thetas_i[i_interference], linestyle="--", color=flatui[i_interference+2], label="Interference angle");
ax.set_xlim(-90, 90); ax.set_ylim(-15, 10)
ax.set_xlabel(r"$\theta$ [degree]"); ax.set_ylabel("Beam pattern [dB]");
ax.set_title("LCMV TF average beam pattern")
ax.legend();

In [ ]:
# Null-controlling beam pattern
nc_tf_beampattern = np.zeros((n_fft_bins, len(theta_grid), len(phi_grid)), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    for i_theta in range(len(theta_grid)):
        for i_phi in range(len(phi_grid)):
            scanning_steering_vector = steering_vectors[i_fft_bin][i_theta][i_phi]
            nc_tf_beampattern[i_fft_bin][i_theta][i_phi] = nc_tf_beamformers[i_fft_bin].transpose().conjugate().dot(
            scanning_steering_vector)

# mvdr_tf_beampattern_normalized = np.abs(mvdr_tf_beampattern)/np.max(np.abs(mvdr_tf_beampattern))
nc_tf_beampattern_db = 20*np.log10(np.abs(nc_tf_beampattern));

fig = plt.figure(figsize=(9, 6)); ax = fig.add_subplot(111)
i_frequencies = [10, 30, 50]
for i_f in i_frequencies:
    ax.plot(theta_grid, nc_tf_beampattern_db[i_f, :, 0], label="{:.0f} Hz".format(i_f/n_fft_bins*sampling_frequency));
ax.axvline(x=theta_s[0], linestyle="--", color=flatui[1], label="Source angle");
for i_interference in range(0, len(thetas_i)):
    ax.axvline(x=thetas_i[i_interference], linestyle="--", color=flatui[i_interference+2], label="Interference angle");
ax.set_xlim(-90, 90);
ax.set_xlabel(r"$\theta$ [degree]"); ax.set_ylabel("Beam pattern [dB]");
ax.set_title("Null-controlling TF beam patterns")
ax.legend();

nc_ave_beampattern = np.mean(np.abs(nc_tf_beampattern[:, :, 0]), axis=0)
nc_ave_beampattern_normalized = nc_ave_beampattern / np.max(nc_ave_beampattern)
# nc_ave_beampattern_normalized = nc_ave_beampattern
nc_ave_beampattern_normalized_db = 20*np.log10(nc_ave_beampattern_normalized)
fig = plt.figure(figsize=(9, 6)); ax = fig.add_subplot(111)
ax.plot(theta_grid, nc_ave_beampattern_normalized_db, label="Beam pattern");
ax.axvline(x=theta_s[0], linestyle="--", color=flatui[1], label="Source angle");
for i_interference in range(0, len(thetas_i)):
    ax.axvline(x=thetas_i[i_interference], linestyle="--", color=flatui[i_interference+2], label="Interference angle");
ax.set_xlim(-90, 90); ax.set_ylim(-15, 1)
ax.set_xlabel(r"$\theta$ [degree]"); ax.set_ylabel("Beam pattern [dB]");
ax.set_title("Null-controlling TF average beam pattern")
ax.legend();

In [ ]:
# Null-controlling DL beam pattern
nc_dl_tf_beampattern = np.zeros((n_fft_bins, len(theta_grid), len(phi_grid)), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    for i_theta in range(len(theta_grid)):
        for i_phi in range(len(phi_grid)):
            scanning_steering_vector = steering_vectors[i_fft_bin][i_theta][i_phi]
            nc_dl_tf_beampattern[i_fft_bin][i_theta][i_phi] = nc_dl_tf_beamformers[i_fft_bin].transpose().conjugate().dot(
            scanning_steering_vector)

# mvdr_tf_beampattern_normalized = np.abs(mvdr_tf_beampattern)/np.max(np.abs(mvdr_tf_beampattern))
nc_dl_tf_beampattern_db = 20*np.log10(np.abs(nc_dl_tf_beampattern));

fig = plt.figure(figsize=(9, 6)); ax = fig.add_subplot(111)
i_frequencies = [10, 30, 50]
for i_f in i_frequencies:
    ax.plot(theta_grid, nc_dl_tf_beampattern_db[i_f, :, 0], label="{:.0f} Hz".format(i_f/n_fft_bins*sampling_frequency));
ax.axvline(x=theta_s[0], linestyle="--", color=flatui[1], label="Source angle");
for i_interference in range(0, len(thetas_i)):
    ax.axvline(x=thetas_i[i_interference], linestyle="--", color=flatui[i_interference+2], label="Interference angle");
ax.set_xlim(-90, 90);
ax.set_xlabel(r"$\theta$ [degree]"); ax.set_ylabel("Beam pattern [dB]");
ax.set_title("Null-controlling DL TF beam patterns")
ax.legend();

nc_dl_ave_beampattern = np.mean(np.abs(nc_dl_tf_beampattern[:, :, 0]), axis=0)
nc_dl_ave_beampattern_normalized = nc_dl_ave_beampattern / np.max(nc_dl_ave_beampattern)
nc_ave_beampattern_normalized = nc_dl_ave_beampattern
nc_dl_ave_beampattern_normalized_db = 20*np.log10(nc_dl_ave_beampattern_normalized)
fig = plt.figure(figsize=(9, 6)); ax = fig.add_subplot(111)
ax.plot(theta_grid, nc_dl_ave_beampattern_normalized_db, label="Beam pattern");
ax.axvline(x=theta_s[0], linestyle="--", color=flatui[1], label="Source angle");
for i_interference in range(0, len(thetas_i)):
    ax.axvline(x=thetas_i[i_interference], linestyle="--", color=flatui[i_interference+2], label="Interference angle");
ax.set_xlim(-90, 90); ax.set_ylim(-15, 1)
ax.set_xlabel(r"$\theta$ [degree]"); ax.set_ylabel("Beam pattern [dB]");
ax.set_title("Null-controlling DL TF average beam pattern")
ax.legend();

In [ ]:
# MPDR beam pattern
mpdr_tf_beampattern = np.zeros((n_fft_bins, len(theta_grid), len(phi_grid)), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    for i_theta in range(len(theta_grid)):
        for i_phi in range(len(phi_grid)):
            scanning_steering_vector = steering_vectors[i_fft_bin][i_theta][i_phi]
            mpdr_tf_beampattern[i_fft_bin][i_theta][i_phi] = mpdr_tf_beamformers[i_fft_bin].transpose().conjugate().dot(
            scanning_steering_vector)

# mvdr_tf_beampattern_normalized = np.abs(mvdr_tf_beampattern)/np.max(np.abs(mvdr_tf_beampattern))
mpdr_tf_beampattern_db = 20*np.log10(np.abs(mpdr_tf_beampattern));

fig = plt.figure(figsize=(9, 6)); ax = fig.add_subplot(111)
i_frequencies = [10, 20, 30, 40, 50]
for i_f in i_frequencies:
    ax.plot(theta_grid, mpdr_tf_beampattern_db[i_f, :, 0], label="{:.0f} Hz".format(i_f/n_fft_bins*sampling_frequency));
ax.axvline(x=theta_s[0], linestyle="--", color=flatui[1], label="Source angle");
for i_interference in range(0, len(thetas_i)):
    ax.axvline(x=thetas_i[i_interference], linestyle="--", color=flatui[i_interference+2], label="Interference angle");
ax.set_xlim(-90, 90);
ax.set_xlabel(r"$\theta$ [degree]"); ax.set_ylabel("Beam pattern [dB]");
ax.set_title("MPDR TF beam patterns")
ax.legend();

mpdr_ave_beampattern = np.mean(np.abs(mpdr_tf_beampattern[:, :, 0]), axis=0)
mpdr_ave_beampattern_normalized = mpdr_ave_beampattern / np.max(mpdr_ave_beampattern)
mpdr_ave_beampattern_normalized_db = 20*np.log10(mpdr_ave_beampattern_normalized)
fig = plt.figure(figsize=(9, 6)); ax = fig.add_subplot(111)
ax.plot(theta_grid, mpdr_ave_beampattern_normalized_db, label="Beam pattern");
ax.axvline(x=theta_s[0], linestyle="--", color=flatui[1], label="Source angle");
for i_interference in range(0, len(thetas_i)):
    ax.axvline(x=thetas_i[i_interference], linestyle="--", color=flatui[i_interference+2], label="Interference angle");
ax.set_xlim(-90, 90); ax.set_ylim(-15, 1)
ax.set_xlabel(r"$\theta$ [degree]"); ax.set_ylabel("Beam pattern [dB]");
ax.set_title("MPDR TF average beam pattern")
ax.legend();

### Steered response

###### DS beamformers aim to emphasize target angles and not to create nulls.

In [ ]:
# Delay-sum steered response
ds_tf_S = np.zeros((n_fft_bins, len(theta_grid), len(phi_grid)), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    sample_tf_covariance_matrix = 1./n_test_samples * ( received_stft_multichannel[i_fft_bin].dot(received_stft_multichannel[i_fft_bin].transpose().conjugate()) )
    for i_theta in range(len(theta_grid)):
        for i_phi in range(len(phi_grid)):
            scanning_steering_vector = steering_vectors[i_fft_bin][i_theta][i_phi]
            ds_tf_S[i_fft_bin][i_theta][i_phi] = (1./n_mics * scanning_steering_vector).transpose().conjugate().dot(
                sample_tf_covariance_matrix).dot(1./n_mics * scanning_steering_vector)

ds_tf_S_normalized = np.abs(ds_tf_S)/np.max(np.abs(ds_tf_S))
ds_tf_S_db = 10*np.log10(ds_tf_S_normalized);

fig = plt.figure(figsize=(9, 6)); ax = fig.add_subplot(111)
i_frequencies = [10, 20, 30]
for i_f in i_frequencies:
    ax.plot(theta_grid, ds_tf_S_db[i_f, :, 0], label="Steered response");
ax.axvline(x=theta_s[0], linestyle="--", color=flatui[1], label="Source angle");
for i_interference in range(0, len(thetas_i)):
    ax.axvline(x=thetas_i[i_interference], linestyle="--", color=flatui[i_interference+2], label="Interference angle");
ax.set_xlim(-90, 90);
ax.set_xlabel(r"$\theta$ [degree]"); ax.set_ylabel("Beam pattern [dB]");
ax.set_title("Delay-sum TF steered response")
ax.legend();

###### MVDR places nulls at interference directions

In [ ]:
# MVDR steered response
mvdr_dl_tf_S = np.zeros((n_fft_bins, len(theta_grid), len(phi_grid)), dtype=np.complex64)
for i_fft_bin in range(n_fft_bins):
    sample_tf_covariance_matrix = 1./n_test_samples * ( interference_stft_multichannel[i_fft_bin].dot(interference_stft_multichannel[i_fft_bin].transpose().conjugate()) )
    for i_theta in range(len(theta_grid)):
        for i_phi in range(len(phi_grid)):
            scanning_steering_vector = steering_vectors[i_fft_bin][i_theta][i_phi]
            mvdr_dl_tf_S[i_fft_bin][i_theta][i_phi] = 1 / ((scanning_steering_vector).transpose().conjugate().dot(
                sample_tf_covariance_matrix).dot(scanning_steering_vector))

mvdr_tf_S_normalized = np.abs(mvdr_dl_tf_S)/np.max(np.abs(mvdr_dl_tf_S))
mvdr_tf_S_db = 10*np.log10(mvdr_tf_S_normalized);

fig = plt.figure(figsize=(9, 6)); ax = fig.add_subplot(111)
i_frequencies = [10, 20, 30]
for i_f in i_frequencies:
    ax.plot(theta_grid, mvdr_tf_S_db[i_f, :, 0], label="Steered response");
ax.axvline(x=theta_s[0], linestyle="--", color=flatui[1], label="Source angle");
for i_interference in range(0, len(thetas_i)):
    ax.axvline(x=thetas_i[i_interference], linestyle="--", color=flatui[i_interference+2], label="Interference angle");
ax.set_xlim(-90, 90);
ax.set_xlabel(r"$\theta$ [degree]"); ax.set_ylabel("Beam pattern [dB]");
ax.set_title("MVDR TF steered response")
ax.legend();